In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch 
from tqdm import tqdm
from scipy import stats
from statsmodels.stats.multitest import multipletests
import mne
import imageio
from collect_data import *
from collect_metrics import get_topomap

In [2]:
stimuli_path = meg_path + '/stimuli/audio'
patient = ['01']
sub_decim = 1
brain_signal_data = []
audio_signal_data = []

In [ ]:
for subject in tqdm(patient):
    print('PATIENT: ', subject)
    for sess in range(len(session)):
        print("SESSION: ", session[sess])
        for story in task_list:
            print('AUDIO_NAME: ', story)
            selected_sound_ids = tasks_with_sound_ids[story]
            story_uid = int(task[story])
            print("STORY_UID: ", story_uid)
            raw = get_bids_raw(meg_path, subject, session[sess], str(story_uid))
            for z, sound_id in enumerate(selected_sound_ids):
                print("SOUND_ID: ", float(sound_id))
                epochs_data = get_epochs(raw, float(story_uid), float(sound_id), sub_decim)
                # save_data(meg_spectr_ranged, 'megsp', subject, str(i), str(story_uid), audio_name, str(z))
                if subject == '01':
                    audio_path = os.path.join(stimuli_path, f'{story}_{z}.wav')
                    audio_spectr = get_audio_spectrogram(audio_path, epochs_data)
                    print('AUDIO_SPECTR_SHAPE: ', audio_spectr.shape)
                    audio_signal_data.append(audio_spectr)
                    save_data(audio_spectr, 'audio_nfft', 'audio', '_', str(story_uid), story, str(z))

In [3]:
megsp_path = os.path.join(meg_path, 'collect_data/megsp')
audio_path = os.path.join(meg_path, 'collect_data/audio_nfft')
megsp_list = os.listdir(megsp_path)
audio_list = os.listdir(audio_path)

select_subj = "02"
megsp_list_session_0 = [f for f in megsp_list if f.startswith(select_subj) and f.split('_')[1] == '0']
megsp_list_session_1 = [f for f in megsp_list if f.startswith(select_subj) and f.split('_')[1] == '1']

audio_tensor_train, audio_tensor_valid, audio_tensor_test = get_splitted_tensor(audio_list, audio_path)
audio_tensor_train = torch.cat((audio_tensor_train, audio_tensor_train), 0)
audio_tensor_valid = torch.cat((audio_tensor_valid, audio_tensor_valid), 0)
audio_tensor_test = torch.cat((audio_tensor_test, audio_tensor_test), 0)
print('DIMENSION_AUDIO_TENSOR_TRAIN: ', audio_tensor_train.shape)
print('DIMENSION_AUDIO_TENSOR_VALID: ', audio_tensor_valid.shape)
print('DIMENSION_AUDIO_TENSOR_TEST: ', audio_tensor_test.shape)

meg_0_tensor_train, meg_0_tensor_valid, meg_0_tensor_test = get_splitted_tensor(megsp_list_session_0, megsp_path)
meg_1_tensor_train, meg_1_tensor_valid, meg_1_tensor_test = get_splitted_tensor(megsp_list_session_1, megsp_path)
meg_tensor_train = torch.cat((meg_0_tensor_train, meg_1_tensor_train), 0)
meg_tensor_valid = torch.cat((meg_0_tensor_valid, meg_1_tensor_valid), 0)
meg_tensor_test = torch.cat((meg_0_tensor_test, meg_1_tensor_test), 0)
print('DIMENSION_MEG_TENSOR_TRAIN: ', meg_tensor_train.shape)
print('DIMENSION_MEG_TENSOR_VALID: ', meg_tensor_valid.shape)
print('DIMENSION_MEG_TENSOR_TEST: ', meg_tensor_test.shape)

DIMENSION_AUDIO_TENSOR_TRAIN:  torch.Size([11958, 4097, 24])
DIMENSION_AUDIO_TENSOR_VALID:  torch.Size([1684, 4097, 24])
DIMENSION_AUDIO_TENSOR_TEST:  torch.Size([3480, 4097, 24])
DIMENSION_MEG_TENSOR_TRAIN:  torch.Size([11958, 208, 16, 26])
DIMENSION_MEG_TENSOR_VALID:  torch.Size([1684, 208, 16, 26])
DIMENSION_MEG_TENSOR_TEST:  torch.Size([3480, 208, 16, 26])


In [7]:
path_to_save = '/srv/nfs-data/sisko/matteoc/meg/audio_stft'
print(os.path.join(path_to_save, 'meg_prediction_ridge_'+select_subj+'.pt'))

/srv/nfs-data/sisko/matteoc/meg/audio_stft/meg_prediction_ridge_02.pt


In [ ]:
from himalaya.ridge import RidgeCV
from himalaya.kernel_ridge import KernelRidgeCV
from himalaya.backend import set_backend


X_train=audio_tensor_train.cpu().numpy().reshape(audio_tensor_train.shape[0], -1)   
y_train=meg_tensor_train.numpy().reshape(meg_tensor_train.shape[0], -1)

X_test=audio_tensor_test.cpu().numpy().reshape(audio_tensor_test.shape[0], -1)  
y_test=meg_tensor_test.numpy().reshape(meg_tensor_test.shape[0], -1)

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=42, shuffle=True)

device_id = 1  # Change this to your desired GPU index
torch.cuda.set_device(device_id)  # Set the current device

backend = set_backend("torch_cuda")
X_train = backend.asarray(X_train).to(f'cuda:{device_id}')
y_train = backend.asarray(y_train).to(f'cuda:{device_id}')

X_test = backend.asarray(X_test).to(f'cuda:{device_id}')
y_test = backend.asarray(y_test).to(f'cuda:{device_id}')

# for i in tqdm(range(208)):
vm=KernelRidgeCV(alphas=[0.01,1,10,1e2,5e3])
vm.fit(X_train,y_train)
predict=vm.predict(X_test)
voxels_scores=vm.score(X_test,y_test)
predict=predict.reshape(X_test.shape[0], 208, -1)
y_test_true=y_test.reshape(y_test.shape[0], 208, -1)



In [ ]:
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.metrics import mean_squared_error

pred_target = []
mse_scores = []
real_target = []
audio_train = audio_tensor_train.reshape(audio_tensor_train.shape[0], -1)
audio_test = audio_tensor_test.reshape(audio_tensor_test.shape[0], -1)

for channel in tqdm(range(num_channel)):    # 10 canali --> tempo +/- 12 minuti
    y_train = meg_tensor_train[:, channel, :, :].reshape(meg_tensor_train.shape[0], -1)
    y_test = meg_tensor_test[:, channel, :, :].reshape(meg_tensor_test.shape[0], -1)

    # model = RidgeCV(alphas=(5000), cv=(4))
    model = Ridge(alpha=5000, max_iter=1000)
    model.fit(audio_train, y_train)

    y_pred = model.predict(audio_test)
    mse = mean_squared_error(y_test, y_pred)
    pred_target.append(y_pred)
    real_target.append(y_test)
    mse_scores.append(mse)
    print(mse_scores)